In [1]:
# Import Packages
import pandas as pd, numpy as np
import os, sys, glob, re
from pathlib import Path
from itertools import compress

In [2]:
exactmatch_yes_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.3_exact_matches\20210101-20220617\exactmatch_yes.csv")
fuzzymatch_filledin_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.4_fuzzy_matches\20210101-20220617\fuzzymatch_manually_filled_in.xlsx")

In [4]:
# Import files
exactmatch = pd.read_csv(exactmatch_yes_filepath)
fuzzymatch = pd.read_excel(fuzzymatch_filledin_filepath)

In [5]:
exactmatch

,clean_firm_name,count,gvkey,company_name,hqcountry
0,CAMDEN PROPERTY TRUST,41,28629.0,Camden Property Trust,US
1,BUNGE LTD,34,144435.0,Bunge Limited,US
2,ROYALTY PHARMA PLC,34,36474.0,Royalty Pharma plc,US
3,SUNRUN INC,32,24905.0,Sunrun Inc,US
4,SUNNOVA ENERGY INTL INC,32,35395.0,Sunnova Energy International Inc.,US
...,...,...,...,...,...
1674,IMAX CORP,1,30312.0,IMAX Corporation,CA
1675,IMMOFINANZ AG,1,211443.0,IMMOFINANZ AG,AT
1676,IMPACT HEALTHCARE REIT PLC,1,323661.0,Impact Healthcare REIT PLC,GB
1677,IMPERIAL METALS CORP,1,20500.0,Imperial Metals Corp,CA


In [6]:
fuzzymatch

,clean_firm_name,count,choice_1_name,choice_1_score,choice_2_name,choice_2_score,choice_3_name,choice_3_score,best_choice
0,EUROPEAN RELIANCE GENERAL INSURANCE CO S A,1,EUROPEAN RELIANCE GENERAL INSURANCE CO SA,98.795181,A E PLASTIK PAK INC,85.500000,A M FOOD SERVICES INC,85.500000,NaN
1,GENWORTH MORTGAGE INSURANC AUSTRALIA LTD,1,GENWORTH MORTGAGE INSURANCE AUSTRALIA LTD,98.765432,AUSTRALIA,90.000000,CB AUSTRALIA LTD,87.096774,NaN
2,CALUMET SPECIALTY PRODUCTS PARTNERS L P,1,CALUMET SPECIALTY PRODUCTS PARTNERS LP,98.701299,ALLIED PRODUCTS CORP,85.500000,ALLOY METAL PRODUCTS INC,85.500000,NaN
3,BROOKFIELD INFRASTRUCTURE PARTNERS L P,10,BROOKFIELD INFRASTRUCTURE PARTNERS LP,98.666667,BROOKFIELD INFRASTRUCTURE CORP,86.363636,B P OIL GAS LTD,85.500000,NaN
4,SHIN KONG FINANCIAL HOLDINGS CO LTD,2,SHIN KONG FINANCIAL HOLDING CO LTD,98.550725,ZB FINANCIAL HOLDINGS LTD,88.936170,SINOPAC FINANCIAL HOLDINGS CO LTD,88.235294,NaN
...,...,...,...,...,...,...,...,...,...
292,GRIFFIN INDUSTRIAL REALTY INC,1,A E PLASTIK PAK INC,85.500000,AA IMPORTING CO INC,85.500000,ABA INDUSTRIES INC,85.500000,NaN
293,INTL MEAL CO HOLDINGS SA,1,SOLIGOR INTL INC,85.500000,AMCA INTL CORP,85.500000,ANR PIPELINE CO,85.500000,NaN
294,ERAMETSLN,1,ERAMET SA,77.777778,ESR,72.000000,VELOCITYSHARES DAILY INVERSE VIX SHORTTERM ETN,67.500000,NaN
295,ZEBU,9,ZEBRA TECHNOLOGIES CORP,77.142857,EBUX INC,77.142857,ZEBALLOS MINING CO,77.142857,NaN


In [ ]:
# get the correct clean_firm_name from compustathassan based on the choice ('company_name')
# merge to get the gvkey,company_name, hqcountry
# save that.

In [ ]:
# Then finally append gvkey and country to the firm names